In [1]:
from graph_tool import Graph, Edge, Vertex
import polars as pl

In [2]:
g: Graph = Graph(directed=True)

In [4]:
df_users: pl.DataFrame = pl.read_parquet("../collection/users.parquet")
df_questions: pl.DataFrame = pl.read_parquet("../collection/questions.parquet")
df_answers: pl.DataFrame = pl.read_parquet("../collection/answers.parquet")

In [11]:
(shape_users, shape_questions, shape_answers) = (
    df_users.shape,
    df_questions.shape,
    df_answers.shape,
)
print(f"Users: {shape_users}, Questions: {shape_questions}, Answers: {shape_answers}")
print(
    f"Total number of nodes: {shape_users[0] + shape_questions[0] + shape_answers[0]}"
)

Users: (1893, 28), Questions: (263, 39), Answers: (1909, 26)
Total number of nodes: 4065


In [5]:
df_users.head()

about_me,accept_rate,account_id,age,answer_count,badge_counts,creation_date,display_name,down_vote_count,is_employee,last_access_date,last_modified_date,link,location,profile_image,question_count,reputation,reputation_change_day,reputation_change_month,reputation_change_quarter,reputation_change_week,reputation_change_year,timed_penalty_date,up_vote_count,user_id,user_type,view_count,website_url
str,i64,i64,i64,i64,struct[3],date,str,i64,bool,date,date,str,str,str,i64,i64,i64,i64,i64,i64,i64,date,i64,i64,cat,i64,str
"""<p>Author of <…",86,11683,null,35683,"{9237,873,9186}",2008-09-26,"""Jon Skeet""",8331,false,2024-02-07,2024-01-31,"""https://stacko…","""Reading, Unite…","""https://www.gr…",55,1447238,10,1000,6160,520,6160,null,17259,22656,"""registered""",2283688,"""http://csharpi…"
"""<p>Author of <…",86,11683,null,35683,"{9237,873,9186}",2008-09-26,"""Jon Skeet""",8331,false,2024-02-07,2024-01-31,"""https://stacko…","""Reading, Unite…","""https://www.gr…",55,1447238,10,1000,6160,520,6160,null,17259,22656,"""registered""",2283688,"""http://csharpi…"
"""<p>You should …",94,335069,null,643,"{371,56,330}",2011-03-16,"""nawfal""",899,false,2023-12-14,2023-11-07,"""https://stacko…","""India""","""https://www.gr…",223,71209,0,50,276,50,276,null,30264,661933,"""registered""",14536,""""""
"""<p>I write cod…",50,3990,null,99,"{41,3,38}",2008-09-11,"""Astra""",6,false,2023-01-18,2023-01-25,"""https://stacko…","""Mesquite, TX, …","""https://www.gr…",6,10965,0,20,80,0,80,null,526,5862,"""registered""",996,"""http://silicon…"
"""<p>Developer w…",50,147914,null,26,"{26,3,12}",2010-06-05,"""iTSrAVIE""",13,false,2018-10-17,2019-11-13,"""https://stacko…","""Mumbai, Mahara…","""https://i.stac…",14,846,0,0,0,0,0,null,103,359349,"""registered""",200,""""""


From a user we can identity the following interesting information:

- The user's Account ID: `account_id`
  - This is very important because it is the unique identifier for the user ACCROSS the entire Stack Exchange network.
- The user's ID: `user_id`
  - This is the unique identifier for the user on the specific site (here, Stack Overflow).
- The number of answers the user has given: `answer_count`
- The number of questions the user has asked: `question_count`
- The number of upvotes the user has received: `up_vote_count`
- The number of downvotes the user has received: `down_vote_count`
- The rate of the user's answers being accepted: `accept_rate`
- The user's location: `location`
- The users's reputation: `reputation`


In [6]:
df_questions.head()

accepted_answer_id,answer_count,answers,body,body_markdown,bounty_amount,bounty_closes_date,bounty_user,close_vote_count,closed_date,closed_details,closed_reason,comment_count,comments,community_owned_date,content_license,creation_date,delete_vote_count,down_vote_count,favorite_count,is_answered,last_activity_date,last_edit_date,last_editor,link,locked_date,migrated_from,migrated_to,notice,owner,protected_date,question_id,reopen_vote_count,score,share_link,tags,title,up_vote_count,view_count
i64,i64,list[struct[26]],str,str,i64,date,struct[1],i64,date,struct[1],str,i64,list[struct[13]],date,str,date,i64,i64,i64,bool,date,date,struct[8],str,date,struct[1],struct[1],struct[3],struct[8],date,i64,i64,i64,str,list[str],str,i64,i64
247623,13,"[{247623,null,null,null,null,null,null,null,null,2008-10-29,""CC BY-SA 4.0"",2008-10-29,null,true,2023-12-12,2023-12-12,{null,{null,null,null},null,null,null,null,null,null},null,null,{86,{9237,873,9186},""Jon Skeet"",""https://stackoverflow.com/users/22656/jon-skeet"",""https://www.gravatar.com/avatar/6d8ebb117e8d83d74ea95fbdd0f87e13?s=256&d=identicon&r=PG"",1447238,22656,""registered""},247621,3190,null,null,null,null}, {317885,null,null,null,null,null,null,null,null,2008-11-25,""CC BY-SA 4.0"",2008-11-25,null,false,2021-08-28,2021-08-28,{null,{null,null,null},null,null,null,null,null,null},null,null,{50,{41,3,38},""Astra"",""https://stackoverflow.com/users/5862/astra"",""https://www.gravatar.com/avatar/73882fc47c81bccc5924afcb08a404a2?s=256&d=identicon&r=PG"",10965,5862,""registered""},247621,330,null,null,null,null}, … {75858507,null,null,null,null,null,null,null,null,2023-03-27,""CC BY-SA 4.0"",2023-03-27,null,false,2023-03-27,2023-03-27,{null,{null,null,null},null,null,null,null,null,null},null,null,{null,{14,2,6},""Ahmad Adibzad"",""https://stackoverflow.com/users/18582805/ahmad-adibzad"",""https://www.gravatar.com/avatar/26584d07294691563c14f6ac8a249991?s=256&d=identicon&r=PG"",543,18582805,""registered""},247621,2,null,null,null,null}]","""<p>What are th…","""What are the c…",null,null,{null},0,null,{null},null,4,"[{""This is one of a good source to understand everything. <a href=""http://en.wikipedia.org/wiki/C_Sharp_(programming_language)"" rel=""nofollow noreferrer"">en.wikipedia.org/wiki/C_Sharp_(programming_language)</a>"",""This is one of a good source to understand everything. http://en.wikipedia.org/wiki/C_Sharp_(programming_language)"",false,18759064,""CC BY-SA 3.0"",2012-12-02,false,""https://stackoverflow.com/questions/247621/what-are-the-correct-version-numbers-for-c#comment18759064_247621"",{null,{null,null,null},""user725388"",null,null,null,null,""does_not_exist""},247621,""question"",{null,{null,null,null},null,null,null,null,null,null},92}, {""Shouldn&#39;t that second paragraph be in a comment instead of the question, since it&#39;s not part of the question"",""Shouldn't that second paragraph be in a comment instead of the question, since it's not part of the question"",false,70397741,""CC BY-SA 3.0"",2017-01-11,false,""https://stackoverflow.com/questions/247621/what-are-the-correct-version-numbers-for-c#comment70397741_247621"",{85,{106,6,69},""TankorSmash"",""https://stackoverflow.com/users/541208/tankorsmash"",""https://www.gravatar.com/avatar/2dfc2e563d2150994a6e06ef37f9e7be?s=256&d=identicon&r=PG"",12296,541208,""registered""},247621,""question"",{null,{null,null,null},null,null,null,null,null,null},1}, … {""<a href=""https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/configure-language-version"" rel=""nofollow noreferrer"">learn.microsoft.com/en-us/dotnet/csharp/language-reference/&hellip;</a>"",""https://learn.microsoft.com/en-us/dotnet/csharp/language-reference/configure-language-version"",false,111491677,""CC BY-SA 4.0"",2020-07-23,false,""https://stackoverflow.com/questions/247621/what-are-the-correct-version-numbers-for-c#comment111491677_247621"",{null,{11,0,8},""kotpal"",""https://stackoverflow.com/users/2183503/kotpal"",""https://www.grav

From a question we can identity the following interesting information:

- The question's ID: `question_id`
- The user who asked the question: `owner`
  - Note that this is a "shallow user" object, from which we can extract the `user_id` of the user.
- The accepted answer's ID to the question: `accepted_answer_id` (if it exists)
- The number of answers the question has received: `answer_count`
- The number of upvotes the question has received: `up_vote_count`
- The number of downvotes the question has received: `down_vote_count`
- The number of favorites the question has received: `favorite_count`
- The last editor of the question: `last_editor`
  - Note that this is a "shallow user" object, from which we can extract the `user_id` of the user.
- The number of views the question has received: `view_count`
- The question's tags: `tags`
  - Note that this is a list of strings, which can be used to identify the topics of the question.
  - We could potentially link questions with similar tags together.


In [7]:
df_answers.head()

answer_id,awarded_bounty_amount,awarded_bounty_users,body,body_markdown,can_comment,can_flag,comment_count,comments,community_owned_date,content_license,creation_date,down_vote_count,is_accepted,last_activity_date,last_edit_date,last_editor,link,locked_date,owner,question_id,score,share_link,tags,title,up_vote_count
i64,i64,list[struct[8]],str,str,bool,bool,i64,list[struct[13]],date,str,date,i64,bool,date,date,struct[8],str,date,struct[8],i64,i64,str,list[str],str,i64
247623,null,null,"""<h1>C# languag…","""# C# language …",null,false,2,"[{""Not sure when the document was posted, but in 2022 we have <a href=""https://learn.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-version-history"" rel=""nofollow noreferrer"">link</a>"",""Not sure when the document was posted, but in 2022 we have [link](https://learn.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-version-history)"",false,130601612,""CC BY-SA 4.0"",2022-10-05,false,""https://stackoverflow.com/questions/247621/what-are-the-correct-version-numbers-for-c/247623#comment130601612_247623"",{null,{10,0,0},""WLFree"",""https://stackoverflow.com/users/12854214/wlfree"",""https://lh3.googleusercontent.com/a-/AAuE7mBJt3OeV8McjClwodfU6ht2fsdUM0TMDs0oHXxhOg=k-s256"",119,12854214,""registered""},247623,""answer"",{null,{null,null,null},null,null,null,null,null,null},4}, {""@WLFree it was originally created on Oct 29, 2008 at 17:10 by Jon Skeet"",""@WLFree it was originally created on Oct 29, 2008 at 17:10 by Jon Skeet"",false,136886598,""CC BY-SA 4.0"",2023-12-12,false,""https://stackoverflow.com/questions/247621/what-are-the-correct-version-numbers-for-c/247623#comment136886598_247623"",{null,{155,11,119},""Guru Stron"",""https://stackoverflow.com/users/2501279/guru-stron"",""https://i.stack.imgur.com/GO0W9.jpg?s=256&g=1"",118140,2501279,""registered""},247623,""answer"",{null,{10,0,0},""WLFree"",""https://stackoverflow.com/users/12854214/wlfree"",""https://lh3.googleusercontent.com/a-/AAuE7mBJt3OeV8McjClwodfU6ht2fsdUM0TMDs0oHXxhOg=k-s256"",119,12854214,""registered""},0}]",2008-10-29,"""CC BY-SA 4.0""",2008-10-29,3,true,2023-12-12,2023-12-12,"{100,{2918,268,2584},""Marc Gravell"",""https://stackoverflow.com/users/23354/marc-gravell"",""https://i.stack.imgur.com/CrVFH.png?s=256&g=1"",1038539,23354,""registered""}","""https://stacko…",null,"{86,{9237,873,9186},""Jon Skeet"",""https://stackoverflow.com/users/22656/jon-skeet"",""https://www.gravatar.com/avatar/6d8ebb117e8d83d74ea95fbdd0f87e13?s=256&d=identicon&r=PG"",1447238,22656,""registered""}",247621,3190,"""https://stacko…",[],"""What are the c…",3193
38506668,null,null,"""<p>This is the…","""This is the sa…",null,false,3,"[{""@Dai, There is no implication in my answer that C# 8.0 is tied to .NET 4.8 In fact there was never a requirement that certain compiler versions required certain .NET versions absolutely, except for select few features. That was true even before C# 8.0. My answer just tracks the timelines of VS, .NET and C# versions. My original answer was in the context of .NET full framework, until that ceased to exist. You can see that there is no other .NET Core versions in the table, because I didnt take the pain to cover that as well. With your edit, viewers will lose information about .NET 4.8."",""@Dai, There is no implication in my answer that C# 8.0 is tied to .NET 4.8 In fact there was never a requirement that certain compiler versions required certain .NET versions absolutely, except for select few features. That was true even before C# 8.0. My answer just tracks the timelines of VS, .NET and C# versions. My original answer was in the context of .NET full framework, until that ceased to exist. You can see that there is no other .NET Core versions in the table, because I didnt take the pain to cover that as well. With your edit, viewers will lose information about .NET 4.8."",false,117329525,""CC BY-SA 4.0"",2021-02-25,false,""https://stackoverflow.com/questions/247621/what-are-the-correct-version-numbers-for-

From an answer we can identity the following interesting information:

- The answer's ID: `answer_id`
- The user who answered the question: `owner`
  - Note that this is a "shallow user" object, from which we can extract the `user_id` of the user.
- The question the answer is answering: `question_id`
- The number of upvotes the answer has received: `up_vote_count`
- The number of downvotes the answer has received: `down_vote_count`
- The number of comments the answer has received: `comment_count`
- The answer's score: `score`
- The answer's tags: `tags`
  - Note that this is a list of strings, which can be used to identify the topics of the answer.
  - We could potentially link answers with similar tags together.
- The last editor of the answer: `last_editor`
  - Note that this is a "shallow user" object, from which we can extract the `user_id` of the user.
